# AI Shopping Assistant 
Using AI algorithms to improve the shopping experience. 

## Description of the agent 
1. **Performance :** Reduction in price/Shopping within the budget
2. **Environment :** Web-based agent. Parses internet product catalogs. 
3. **Actuators :** Returns a curated shopping list containing the items 
4. **Sensors :** Checks for the price and rating (extras : ETA,product reviews), Historical data (User's shopping history)


In [2]:
import numpy as np
import pandas as pd

# What does the agent do ? 
1. Takes e-commerce website data/small simulated product dataset.
2. Takes user preferences for items to be ordered every week
3. Returns a curated shopping list, reducing price and maximizing ratings



In [3]:
data = pd.read_csv('product_catalog.csv')
data

,ProductID,ProductName,RetailPrice,DiscountedPrice,Rating,Category
0,1,Lime Pickle Brand 1,225,185,3.5,"Food, Pickle"
1,2,Citron Pickle,250,225,4.0,"Food, Pickle"
2,3,Mixed Pickle,230,175,4.5,"Food, Pickle"
3,4,Lime Pickle Brand 2,240,190,3.9,"Food, Pickle"
4,5,Broom Brand 1,500,455,3.8,"Household, Broom"
5,6,Broom Brand 2,550,490,3.9,"Household, Broom"
6,7,Broom Brand 3,634,422,3.6,"Household, Broom"
7,8,Floor Cleaner Brand 1,650,450,2.8,"Household, Floor Cleaner"
8,9,Floor Cleaner Brand 2,600,435,2.8,"Household, Floor Cleaner"
9,10,Pen pack brand 1,200,150,4.8,"Stationery, Pen"


In [4]:
#Splitting categories and stripping whitespaces
data[['Category','SubCategory']] = data['Category'].str.split(', ', expand=True)
data['SubCategory'] = data['SubCategory'].str.strip()
data

,ProductID,ProductName,RetailPrice,DiscountedPrice,Rating,Category,SubCategory
0,1,Lime Pickle Brand 1,225,185,3.5,Food,Pickle
1,2,Citron Pickle,250,225,4.0,Food,Pickle
2,3,Mixed Pickle,230,175,4.5,Food,Pickle
3,4,Lime Pickle Brand 2,240,190,3.9,Food,Pickle
4,5,Broom Brand 1,500,455,3.8,Household,Broom
5,6,Broom Brand 2,550,490,3.9,Household,Broom
6,7,Broom Brand 3,634,422,3.6,Household,Broom
7,8,Floor Cleaner Brand 1,650,450,2.8,Household,Floor Cleaner
8,9,Floor Cleaner Brand 2,600,435,2.8,Household,Floor Cleaner
9,10,Pen pack brand 1,200,150,4.8,Stationery,Pen


In [5]:
user_history = pd.read_csv('user_history.csv')
user_history

,UserID,Product,Paid,ProductID
0,111,Lime Pickle Brand 1,185,1
1,111,Broom Brand 2,490,6
2,111,Floor Cleaner Brand 1,450,8
3,112,Pen Pack Brand 1,150,10
4,112,Broom Brand 3,422,7


In [6]:
#User order - contains id, items and total budget
user_order_info = [111, ['Pickle', 'Pen'], 500]

#Merging dataframes based on the product id key
history = pd.merge(data,user_history,on='ProductID')

We will use the following conditions to curate a new shopping list 
1. If an item from a category has already been bought - Check if it is within the budget after purchasing all other items 
2. If not choose based on a distance metric 
3. Check if the list meets the budget

Here i've chosen a simple metric (-i + j*100) where i is the product price and product ratings.

To make things simple, we will assume the customer buys on 1 item in each sub-category.

In [23]:
def check_budget(budget, shopping_list):
  total = 0
  for _,price in shopping_list:
    total += price
  if total > budget:
    return False
  return True 

In [32]:
def curate_shopping_list(items_to_order, products_history):
  #Shopping list curation
  shopping_list = []
  for item in items_to_order:
    #Check if item was previously ordered, choose same item and add to the list
    if item in products_history['SubCategory'].values:
      product_id = products_history[products_history['SubCategory']==item]['ProductID'].values[0]
      product_name = products_history[products_history['SubCategory']==item]['ProductName'].values[0]
      product_price = products_history[products_history['SubCategory']==item]['DiscountedPrice'].values[0]
      shopping_list.append((product_name, product_price))
    else:
      #Create a list of all product price and ratings in the same category 
      product_prices = data[data['SubCategory']==item]['DiscountedPrice']
      product_ratings = data[data['SubCategory']==item]['Rating']
      #Defining metric to choose a product
      product_metric = [-i+(j*100) for i,j in zip(product_prices, product_ratings)]
      index = product_metric.index(max(product_metric))
      #Choosing product based on the metric
      chosen_product = data[data['SubCategory']==item].iloc[index]
      shopping_list.append((chosen_product['ProductName'], chosen_product['DiscountedPrice']))
  return shopping_list

In [33]:
def run_shopping_agent(user_order_info):
  #Extracting values from user orders
  user_id = user_order_info[0]
  items_to_order = user_order_info[1]
  budget = user_order_info[2]

  #Listing previous items bought by the specific user
  if user_id in history['UserID'].values:
    products_history = history[history['UserID']==user_order_info[0]]
  
  shopping_list = curate_shopping_list(items_to_order, products_history)
  if check_budget(budget,shopping_list)==False:
    shopping_list = curate_shopping_list(items_to_order, products_history)
  return shopping_list

In [34]:
run_shopping_agent(user_order_info)

[('Lime Pickle Brand 1', 185), ('Pen pack brand 1', 150)]